In [1]:
import cv2 as cv2
import numpy as np # must first import numpy before import pyspin
import PySpin
from time import sleep
import threading
import os
import sys
import copy

# Hold image sequences
IMAGES = None

In [2]:
class SystemEventHandler(PySpin.InterfaceEventHandler):
    def __init__(self):
        super(SystemEventHandler, self).__init__()
        # self.cameras = None
        # self.system = system
    def OnDeviceArrival(self, serial_number):
        print('Device {} arrival...........'.format(serial_number))
        print('相机 %s 已经连接成功' % serial_number)
        
    def OnDeviceRemoval(self, serial_number):
        # callback_stop_this_camera(serial_number)
        print('相机 %s 失去连接!，请检查相机，连接正常后重启系统.....' % serial_number)
        
   

In [1]:
class Camera(object):
    def __init__(self, cam,callbacks):
        self.taskrun = True
        self.cam = cam
        self.image = None
        self.image_list = []
        self.callback = callbacks
        t = threading.Thread(target = self.run,args = (self.cam, self.callback)) 
        t.daemon = True
        t.start()
        

    def run(self, cam, callbacks):
        print("start while loop........")
        while self.taskrun: #self.start_acquire_image: #not stop_event:
            try:
                self.image = cam.GetNextImage()
                if self.image.IsIncomplete():
                    print('Image incomplete with image status %d ...' % self.image.GetImageStatus())
                    print(self.image.GetTimeStamp())
                    # print("image incoming...................")
                else:
                    '''
                    # Convert to mono8 asignt to kivy global img
                    '''
                    image_converted = self.image.Convert(PySpin.PixelFormat_BayerRG8, PySpin.HQ_LINEAR)
                    img_arr = image_converted.GetNDArray()
                    #img_arr_c = copy.deepcopy(img_arr)
                    #self.count += 1
                    #image_c = copy.deepcopy(image_converted)
                    # call image proccess to fusion imaage
                    if len(self.image_list)< 3:
                        self.image_list.append(img_arr)
                    else:
                        callbacks(self.image_list)
                        self.taskrun = False
                    # clear buffer from camera
                    self.image.Release()
                    print('image getting success....')
            except PySpin.SpinnakerException as ex:
                print('in image event polling images!')
                print(ex)
                

In [4]:
# display to screen
#self.texture.blit_buffer(bytes(config.V_STACK_IMGS), colorfmt='rgb', bufferfmt='ubyte')
def callback(image_list):
    # read image from buffer
    '''
    images = []
    for img in image_list:
        #arr = np.asarray(bytearray(img), dtype=np.uint8)
        
        
        #image = cv2.imdecode(np.frombuffer(arr, np.uint8), cv2.IMREAD_ANYCOLOR)
        image = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        images.append(image)
    print('image number', len(images))
    
    '''
    
    #img_list = [cv.imread(fn) for fn in img_fn]
    images = [cv2.cvtColor(img, cv2.COLOR_BayerRG2BGR) for img in image_list]
    # Exposure fusion using Mertens
    merge_mertens = cv2.createMergeMertens()
    res_mertens = merge_mertens.process(images)
    print('go to end of hdr ....')
    # Convert datatype to 8-bit and show
    global IMAGES
    IMAGES = np.clip(res_mertens*255, 0, 255).astype('uint8')
    


def _change_mono_to_rgb_format(img):
    # combine single gray channel to three channel for 
    # further mark process
    return cv2.merge([img,img,img])

In [5]:
from numpy import true_divide


def main():
    result = True
     # Retrieve singleton reference to system object
    system = PySpin.System.GetInstance()

    # Retrieve list of cameras from the system
    cam_list = system.GetCameras()

    num_cameras = cam_list.GetSize()

    print('Number of cameras detected: %d' % num_cameras)

    # Finish if there are no cameras
    if num_cameras == 0:

        # Clear camera list before releasing system
        cam_list.Clear()

        # Release system instance
        system.ReleaseInstance()

        print('Not enough cameras!')
        input('Done! Press Enter to exit...')
        return False

    # Run example on each camera
    for i, cam in enumerate(cam_list):

        print('Running example for camera %d...' % i)
        cam.Init()
        sleep(3)
        print("sleeping.....")
        cam.BeginAcquisition()
        sleep(1)
        camera = Camera(cam, callback)
        print('Camera %d example running... \n' % i)

    # Release reference to camera
    # NOTE: Unlike the C++ examples, we cannot rely on pointer objects being automatically
    # cleaned up when going out of scope.
    # The usage of del is preferred to assigning the variable to None.
    display = True
    while display:
        if IMAGES is None:
            continue
        else:
            im = cv2.resize(IMAGES, (820, 600))    
            cv2.imshow('Image', im)
            cv2.waitKey(0)
            display = False
    input("press any key stop.....")
    cam.EndAcquisition()
    # Deinitialize camera 
    cam.DeInit()
    del camera
    del cam

    # Clear camera list before releasing system
    cam_list.Clear()

    # Release system instance
    system.ReleaseInstance()  

    #input('Done! Press Enter to exit...')
    return result



In [6]:
#print(cv.__version__)
if __name__ == '__main__':
    if main():
        sys.exit(0)
    else:
        sys.exit(1)


Number of cameras detected: 1
Running example for camera 0...
start while loop........
Camera 0 example running... 

image getting success....
image getting success....
image getting success....
go to end of hdr ....
image getting success....


: 

: 

In [38]:
print(cv2.__version__)

4.5.5
